In [1]:
from sympy import *
from IPython.display import display

def quality_evaluation(starting_tier, target_tier, max_qual, base_prod, prods, quals, debug = False, gradients = False):
    h = 5
    m = target_tier
    #h, m = symbols("h, m", integer=True, positive=True, nonzero=True)
    T4qual = Rational(1, 40) * (Rational(5, 2) if max_qual else Rational(1, 1)) # type: ignore
    T4prod = Rational(1, 10) * (Rational(5, 2) if max_qual else Rational(1, 1)) # type: ignore
    q_cv, p_cv, q_rv, p_rv = quals * T4qual, (1.5 if base_prod else 1.0) + prods * T4prod, 4 * T4qual, 1 # type: ignore
    d_0v = [0, 0, 0, 0, 0]
    d_0v[starting_tier] = 1

    q_c, p_c, q_r, p_r = symbols("q_c, p_c, q_r, p_r", real=True, positive=True)
    q_cs = Matrix([[*symbols(", ".join("q_{c_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])
    p_cs = Matrix([[*symbols(", ".join("p_{c_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])
    q_rs = Matrix([[*symbols(", ".join("q_{r_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])
    p_rs = Matrix([[*symbols(", ".join("p_{r_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])

    #display(q_cs)

    d_0 = Matrix([[*symbols(", ".join("d_0"+str(i) for i in range(1, h+1)), integer=True, positive=True, nonzero=True)]]).T

    l = Matrix([[1]*(m-1)+[0]*(h+1-m)])
    #L = diag(*l)

    full_subs_list = [(x, q_cv) for x in q_cs] + [(x, p_cv) for x in p_cs] + [(x, q_rv) for x in q_rs] + [(x, p_rv) for x in p_rs] + [(k, v) for k, v in zip(d_0, d_0v)]

    i, j, r = symbols("i, j, k", integer=True)

    #C_0f = Piecewise((0, i>j),(p_c-p_c*q_c*(Rational(10, 9) - 10**(1+i-h)/9), Eq(i,j)),(p_c*q_c/(10**(j-i-1)), i<j))
    #R_0f = Piecewise((0, i>j),(p_r/4-p_r/4*q_r*(Rational(10, 9) - 10**(1+i-h)/9), Eq(i,j)),(p_r*q_r/(4*10**(j-i-1)), i<j))
    Cf = Piecewise((0, i>j),(p_c-p_c*q_c*(Rational(10, 9) - 10**(2+i-h)/9), Eq(i,j)),(p_c*q_c/(10**(j-i-1)), i<j))
    Rf = Piecewise((0, i>j),(p_r/4-p_r/4*q_r*(Rational(10, 9) - 10**(2+i-h)/9), Eq(i,j)),(p_r*q_r/(4*10**(j-i-1)), i<j))

    #Cf = piecewise_fold(Piecewise((C_0f, i<m-1),(0, (i>=m-1) & Ne(i,j)),(1, (i>=m-1) & Eq(i,j))))
    #Rf = piecewise_fold(Piecewise((R_0f, i<m-1),(0, (i>=m-1) & Ne(i,j)),(1, (i>=m-1) & Eq(i,j))))

    def apply_column_based_effects(L):
        return Matrix([[L[:,iv].subs([(q_c, q_cs[iv]), (p_c, p_cs[iv]), (q_r, q_rs[iv]), (p_r, p_rs[iv])]) for iv in range(L.shape[1])]])

    C = FunctionMatrix(h, h, Lambda((j, i), Cf)).as_explicit()
    R = FunctionMatrix(h, h, Lambda((j, i), Rf)).as_explicit()
    C = apply_column_based_effects(C)
    R = apply_column_based_effects(R)

    if debug:
        display(C.subs(full_subs_list))
        display(R.subs(full_subs_list))

    #CR = simplify(C * R)
    CR = (C * R)
    T = CR * diag(*l) + eye(h) - diag(*l)

    if debug:
        display(T.subs(full_subs_list))

    def L_eigenvectors(L):
        eigenvectors = []
        for r in range(1, L.shape[0]+1):
            L_minus = L - L[r-1,r-1] * eye(L.shape[0])
            eigenvector = [0] * (r - 1) + [1]
            for jv in range(r+1, L.shape[0]+1):
                s = 0
                for iv in range(1, jv):
                    s += L_minus[jv-1,iv-1] * eigenvector[iv-1]
                if L_minus[jv-1,jv-1]==0:
                    assert s==0
                    eigenvector.append(0)
                else:   
                    eigenvector.append(-s/L_minus[jv-1,jv-1])
            eigenvectors.append(eigenvector)
        return Matrix(eigenvectors).T

    def L_inverse(L):
        columns = []
        for r in range(1, L.shape[0]+1):
            column = [0] * (r - 1) + [1 / L[r-1,r-1]]
            for jv in range(r+1, L.shape[0]+1):
                s = 0
                for iv in range(1, jv):
                    s += L[jv-1,iv-1] * column[iv-1]
                column.append(-s/L[jv-1,jv-1])
            columns.append(column)
        return Matrix(columns).T

    def D_inf_val(D):
        return diag(*[1 if D[iv,iv]==1 else 0 for iv in range(D.shape[0])])

    def D_inf_sum(D):
        #return simplify(diag(*[0 if D[iv,iv]==1 else 1/(1-D[iv,iv]) for iv in range(D.shape[0])]))
        return diag(*[0 if D[iv,iv]==1 else 1/(1-D[iv,iv]) for iv in range(D.shape[0])])

    D = diag(*Matrix.diagonal(T))
    #E = simplify(L_eigenvectors(CR))
    #E_inv = simplify(L_inverse(E))
    E = L_eigenvectors(T)
    E_inv = L_inverse(E)

    #display(E)
    #display(D)
    #display(E_inv)

    d_inf = E * D_inf_val(D) * E_inv * C * d_0
    #display(d_inf)

    L = (E * D_inf_sum(D) * E_inv * C * d_0)


    #display(D_inf_sum(D))

    #subs_list = [(q_c, q_cv), (p_c, p_cv), (q_r, q_rv), (p_r, p_rv)] + [(k, v) for k, v in zip(d_0, d_0v)]

    #display(d_inf.subs(subs_list))
    #print(float(1/d_inf.subs(subs_list)[m-1]))
    #display(Lc.subs(subs_list))
    #print(float(Lc.subs(subs_list)))
    #display(Lr.subs(subs_list))
    #print(float(Lr.subs(subs_list)))

    if debug:
        print("========================================")
        display(d_inf.subs(full_subs_list))
        print(float(1/d_inf.subs(full_subs_list)[m-1]))
        display(L.subs(full_subs_list))
        print(float((l * L)[0,0].subs(full_subs_list)))
    result = float(1/d_inf.subs(full_subs_list)[m-1])

    #derivatives = [q_c, p_c, q_r, p_r]
    full_derivatives = [*q_cs] + [*p_cs] + [*q_rs] + [*p_rs]
    full_derivatives_bundle = list(zip([*q_cs], list(range(q_cs.shape[1])))) + list(zip([*p_cs], list(range(p_cs.shape[1])))) + \
                                         list(zip([*q_rs], list(range(q_rs.shape[1])))) + list(zip([*p_rs], list(range(p_rs.shape[1]))))
    if gradients:
        for x, g in zip(full_derivatives, [d_inf.diff(dt)[m-1,0] / (L+d_0)[i,0] for dt, i in full_derivatives_bundle]):
            print("========================================")
            print(x)
            print(float(g.subs(full_subs_list)))
    
    return result

In [ ]:
quality_evaluation(0, 5, False, False, 0, 4, debug=True)

In [ ]:
quality_evaluation(0, 5, False, False, 4, 0, debug=True)

In [ ]:
output_dump: str = ""
for starting_tier in range(0, 4):
    for target_tier in range(starting_tier+1, 5+1):
        for max_qual in [False, True]:
            for max_assembler_modules in [4, 5]:
                for base_prod in [False, True]:
                    best_module_setup_quals = 0
                    best_module_setup_prods = 0
                    best_result = 10000000000000000000
                    for prods in range(max_assembler_modules+1):
                        quals = max_assembler_modules-prods

                        result = quality_evaluation(starting_tier, target_tier, max_qual, base_prod, prods, quals)

                        if result < best_result:
                            best_result = result
                            best_module_setup_quals = quals
                            best_module_setup_prods = prods
                        
                    print(str(starting_tier+1)+"->"+str(target_tier)+(" with max qual " if max_qual else " at base qual ")+" with "+str(max_assembler_modules)+" modules and "+str(50 if base_prod else 0)+" base productivty:\n"+\
                          "Proper setup is: "+str(best_module_setup_prods)+" productivity modules and "+str(best_module_setup_quals)+" quality modules, resulting in "+str(best_result)+" inputs per 1 output.")
                    output_dump += str(starting_tier+1)+", "+str(target_tier)+", "+str((5 if max_qual else 1))+", "+str(max_assembler_modules)+", "+str((50 if base_prod else 0))+", "+str(best_module_setup_prods)+", "+str(best_module_setup_quals)+", "+str(best_result)+"\n"

In [ ]:
print(output_dump)

In [ ]:
[*q_cs]

In [ ]:
list(range(1, 5))

In [88]:
#newmethod
from sympy import *
from sympy.stats import *
from IPython.display import display


kappa = 5
m = 6
T4qual = Rational(1, 40) * Rational(5, 2) # type: ignore
T4prod = Rational(1, 10) * Rational(5, 2) # type: ignore

q_ms = Matrix([[*symbols(", ".join("q_{m_"+str(i)+"}" for i in range(1, kappa+1)), rational=True, positive=True)]])
q_rs = Matrix([[*symbols(", ".join("q_{r_"+str(i)+"}" for i in range(1, kappa+1)), rational=True, positive=True)]])
p_ms = Matrix([[*symbols(", ".join("p_{m_"+str(i)+"}" for i in range(1, kappa+1)), rational=True, positive=True)]])
p_rs = Matrix([[*symbols(", ".join("p_{r_"+str(i)+"}" for i in range(1, kappa+1)), rational=True, positive=True)]])

MRVs = [FiniteRV('M'+str(i), {j: 1-q_ms[i-1]*(Rational(10,9) - Rational(Pow(10,(1-(kappa-i))),9)) if i==j else q_ms[i-1]*Rational(1,10**(j-i-1)) for j in range(i, kappa+1)}) for i in range(1, kappa+1)] # type: ignore
#RRVs = [FiniteRV('R'+str(i), dict([(0, Rational(3,4))]+[(j, Rational(1,4) * (1-q_rs[i-1]*(Rational(10,9) - Rational(Pow(10,(1-(kappa-i))),9)) if i==j else q_rs[i-1]*Rational(1,10**(j-i-1)))) for j in range(i, kappa+1)])) for i in range(1, kappa+1)] # type: ignore
RRVs  = [FiniteRV('R'+str(i), {j: 1-q_rs[i-1]*(Rational(10,9) - Rational(Pow(10,(1-(kappa-i))),9)) if i==j else q_rs[i-1]*Rational(1,10**(j-i-1)) for j in range(i, kappa+1)}) for i in range(1, kappa+1)] # type: ignore

es = [eye(kappa).col(i) for i in range(kappa)]

binom = Binomial('B', m, Rational(1, 4))

special_subs = [(q_rs[0, i], 4 * T4qual) for i in range(kappa)] + [(p_rs[0, i], 1) for i in range(kappa)] + [(q_ms[0, i], 5 * T4qual) for i in range(kappa)] + [(p_ms[0, i], 1 + Rational(1, 2)) for i in range(kappa-1)]+[(p_ms[0, kappa-1], 1 + Rational(1, 2) + 5 * T4prod)]

N_i_moment_symbol_cache = {}
def N_i_moment_symbols(n: int):
    if not n in N_i_moment_symbol_cache.keys():
        N_i_moment_symbol_cache[n] = [Matrix([*symbols(", ".join("(N_{"+str(i)+"}^{"+str(n)+"})_"+str(j) for j in range(1, kappa+1)), rational=True)]) for i in range(1, kappa+1)]

    return N_i_moment_symbol_cache[n]

def N_i_moment_n_eq(i: int, n: int):
    moment_symbols = N_i_moment_symbols(n)
    s = Matrix.zeros(kappa, 1)

    for j in range(1, kappa):
        r = Matrix.zeros(kappa, 1)
        for k in range(1, kappa+1):
            rv = P(Eq(RRVs[j-1], k)) * moment_symbols[k-1]
            if r == Matrix.zeros(kappa, 1):
                r = rv
            else:
                r += rv

        s += P(Eq(MRVs[i-1], j)) * Pow(p_rs[j-1], n) * r
    
    s *= Pow(p_ms[i-1], n) * moment(binom, n, 0) / Pow(m, n) # type: ignore
    
    s += Pow(p_ms[i-1], n) * P(Eq(MRVs[i-1], kappa)) * es[kappa-1]

    return [Eq(symb, solve(Eq(symb, s_v), symb)[0]) for symb, s_v in zip(moment_symbols[i-1], s)]

def input_N_i_moment_n_eq(i: int, n: int):
    moment_symbols = N_i_moment_symbols(n)
    s = Matrix.zeros(kappa, 1)

    if i==kappa:
        s = es[kappa-1]

    else:
        for j in range(1, kappa):
            r = Matrix.zeros(kappa, 1)
            for k in range(1, kappa+1):
                rv = P(Eq(RRVs[j-1], k)) * moment_symbols[k-1]
                if r == Matrix.zeros(kappa, 1):
                    r = rv
                else:
                    r += rv

            s += P(Eq(MRVs[i-1], j)) * Pow(p_rs[j-1], n) * r
        
        s *= Pow(p_ms[i-1], n) * moment(binom, n, 0) / Pow(m, n) # type: ignore

    return [Eq(symb, solve(Eq(symb, s_v), symb)[0]) for symb, s_v in zip(moment_symbols[i-1], s)]

def N_moment_n_closed_forms(n: int):
    eqs = [N_i_moment_n_eq(i, n) for i in range(1, kappa+1)]
    solutions = {eqs[-1][-1].lhs: eqs[-1][-1].rhs}
    for i in range(kappa-1, -1, -1):
        for j in range(kappa-1, -1, -1):
            solutions[eqs[i][j].lhs] = eqs[i][j].rhs.subs(solutions)

    out = {}
    for k, v in solutions.items():
        if int(str(k)[-1])==kappa:
            out[k] = v
    return out

def input_N_moment_n_closed_forms(n: int):
    eqs = [input_N_i_moment_n_eq(i, n) for i in range(1, kappa+1)]
    solutions = {eqs[-1][-1].lhs: eqs[-1][-1].rhs}
    for i in range(kappa-1, -1, -1):
        for j in range(kappa-1, -1, -1):
            solutions[eqs[i][j].lhs] = eqs[i][j].rhs.subs(solutions)

    out = {}
    for k, v in solutions.items():
        if int(str(k)[-1])==kappa:
            out[k] = v
    return out
    
#display(N_i_moment_symbols(2, 5))
symbs = set()
for N_i_symbs in N_i_moment_symbols(2):
    symbs.update(N_i_symbs.free_symbols)
print(symbs)
for equation in input_N_i_moment_n_eq(3, 1):
    display(equation)
    #print(solve(equation, symbs)[0])
print("==========================")
valuable_first_moments = N_moment_n_closed_forms(1)
#for k, fm in valuable_first_moments.items():
#    display(Eq(k, fm))
valuable_second_moments = N_moment_n_closed_forms(2)
#for k, sm in valuable_second_moments.items():
#    display(Eq(k, sm))
input_valuable_first_moments = input_N_moment_n_closed_forms(1)
input_valuable_second_moments = input_N_moment_n_closed_forms(2)

{(N_{4}^{2})_3, (N_{3}^{2})_1, (N_{2}^{2})_2, (N_{1}^{2})_4, (N_{2}^{2})_1, (N_{1}^{2})_1, (N_{2}^{2})_3, (N_{5}^{2})_4, (N_{3}^{2})_3, (N_{1}^{2})_2, (N_{4}^{2})_1, (N_{1}^{2})_5, (N_{5}^{2})_1, (N_{5}^{2})_3, (N_{2}^{2})_4, (N_{3}^{2})_4, (N_{2}^{2})_5, (N_{5}^{2})_5, (N_{4}^{2})_4, (N_{4}^{2})_5, (N_{3}^{2})_2, (N_{3}^{2})_5, (N_{5}^{2})_2, (N_{4}^{2})_2, (N_{1}^{2})_3}


Eq((N_{3}^{1})_1, p_{m_3}*(-110*(N_{4}^{1})_1*p_{r_3}*q_{m_3}*q_{r_3} + 100*(N_{4}^{1})_1*p_{r_3}*q_{r_3} - 100*(N_{4}^{1})_1*p_{r_4}*q_{m_3}*q_{r_4} + 100*(N_{4}^{1})_1*p_{r_4}*q_{m_3} - 11*(N_{5}^{1})_1*p_{r_3}*q_{m_3}*q_{r_3} + 10*(N_{5}^{1})_1*p_{r_3}*q_{r_3} + 100*(N_{5}^{1})_1*p_{r_4}*q_{m_3}*q_{r_4})/(-121*p_{m_3}*p_{r_3}*q_{m_3}*q_{r_3} + 110*p_{m_3}*p_{r_3}*q_{m_3} + 110*p_{m_3}*p_{r_3}*q_{r_3} - 100*p_{m_3}*p_{r_3} + 400))

Eq((N_{3}^{1})_2, p_{m_3}*(-110*(N_{4}^{1})_2*p_{r_3}*q_{m_3}*q_{r_3} + 100*(N_{4}^{1})_2*p_{r_3}*q_{r_3} - 100*(N_{4}^{1})_2*p_{r_4}*q_{m_3}*q_{r_4} + 100*(N_{4}^{1})_2*p_{r_4}*q_{m_3} - 11*(N_{5}^{1})_2*p_{r_3}*q_{m_3}*q_{r_3} + 10*(N_{5}^{1})_2*p_{r_3}*q_{r_3} + 100*(N_{5}^{1})_2*p_{r_4}*q_{m_3}*q_{r_4})/(-121*p_{m_3}*p_{r_3}*q_{m_3}*q_{r_3} + 110*p_{m_3}*p_{r_3}*q_{m_3} + 110*p_{m_3}*p_{r_3}*q_{r_3} - 100*p_{m_3}*p_{r_3} + 400))

Eq((N_{3}^{1})_3, p_{m_3}*(-110*(N_{4}^{1})_3*p_{r_3}*q_{m_3}*q_{r_3} + 100*(N_{4}^{1})_3*p_{r_3}*q_{r_3} - 100*(N_{4}^{1})_3*p_{r_4}*q_{m_3}*q_{r_4} + 100*(N_{4}^{1})_3*p_{r_4}*q_{m_3} - 11*(N_{5}^{1})_3*p_{r_3}*q_{m_3}*q_{r_3} + 10*(N_{5}^{1})_3*p_{r_3}*q_{r_3} + 100*(N_{5}^{1})_3*p_{r_4}*q_{m_3}*q_{r_4})/(-121*p_{m_3}*p_{r_3}*q_{m_3}*q_{r_3} + 110*p_{m_3}*p_{r_3}*q_{m_3} + 110*p_{m_3}*p_{r_3}*q_{r_3} - 100*p_{m_3}*p_{r_3} + 400))

Eq((N_{3}^{1})_4, p_{m_3}*(-110*(N_{4}^{1})_4*p_{r_3}*q_{m_3}*q_{r_3} + 100*(N_{4}^{1})_4*p_{r_3}*q_{r_3} - 100*(N_{4}^{1})_4*p_{r_4}*q_{m_3}*q_{r_4} + 100*(N_{4}^{1})_4*p_{r_4}*q_{m_3} - 11*(N_{5}^{1})_4*p_{r_3}*q_{m_3}*q_{r_3} + 10*(N_{5}^{1})_4*p_{r_3}*q_{r_3} + 100*(N_{5}^{1})_4*p_{r_4}*q_{m_3}*q_{r_4})/(-121*p_{m_3}*p_{r_3}*q_{m_3}*q_{r_3} + 110*p_{m_3}*p_{r_3}*q_{m_3} + 110*p_{m_3}*p_{r_3}*q_{r_3} - 100*p_{m_3}*p_{r_3} + 400))

Eq((N_{3}^{1})_5, p_{m_3}*(-110*(N_{4}^{1})_5*p_{r_3}*q_{m_3}*q_{r_3} + 100*(N_{4}^{1})_5*p_{r_3}*q_{r_3} - 100*(N_{4}^{1})_5*p_{r_4}*q_{m_3}*q_{r_4} + 100*(N_{4}^{1})_5*p_{r_4}*q_{m_3} - 11*(N_{5}^{1})_5*p_{r_3}*q_{m_3}*q_{r_3} + 10*(N_{5}^{1})_5*p_{r_3}*q_{r_3} + 100*(N_{5}^{1})_5*p_{r_4}*q_{m_3}*q_{r_4})/(-121*p_{m_3}*p_{r_3}*q_{m_3}*q_{r_3} + 110*p_{m_3}*p_{r_3}*q_{m_3} + 110*p_{m_3}*p_{r_3}*q_{r_3} - 100*p_{m_3}*p_{r_3} + 400))

In [90]:
import itertools

def calculate_best_setup(mod_count: int, base_prod: Rational):
    global_subs = [(q_rs[0, i], 4 * T4qual) for i in range(kappa)] + [(p_rs[0, i], 1) for i in range(kappa)]
    reduced = [vfm.subs(global_subs) for vfm in valuable_first_moments.values()]
    reduced_variances = [vsm.subs(global_subs) for vsm in valuable_second_moments.values()]
    best_setups = []
    best_module_values = []
    for i in range(0, kappa):
        best = None
        best_val = 0
        for pc in range(0, mod_count+1):
            val = reduced[i].subs([(p_ms[0, kappa - i - 1], T4prod * pc + base_prod + 1), (q_ms[0, kappa - i - 1], (mod_count - pc) * T4qual)])
            if val >= best_val:
                best = (pc, mod_count - pc)
                best_val = val

        assert not best is None
        reduced = [r.subs([(p_ms[0, kappa - i - 1], T4prod * best[0] + base_prod + 1), (q_ms[0, kappa - i - 1], best[1] * T4qual)]) for r in reduced]
        reduced_variances = [r.subs([(p_ms[0, kappa - i - 1], T4prod * best[0] + base_prod + 1), (q_ms[0, kappa - i - 1], best[1] * T4qual)]) for r in reduced_variances]
        best_setups.insert(0, best)
        best_module_values.insert(0, (T4prod * best[0] + base_prod + 1, best[1] * T4qual))
    
    return reduced, reduced_variances, best_setups, best_module_values

def calculate_best_input_setup(mod_count: int, base_prod: Rational):
    global_subs = [(q_rs[0, i], 4 * T4qual) for i in range(kappa)] + [(p_rs[0, i], 1) for i in range(kappa)]
    reduced = [vfm.subs(global_subs) for vfm in input_valuable_first_moments.values()]
    reduced_variances = [vsm.subs(global_subs) for vsm in input_valuable_second_moments.values()]
    best_setups = []
    best_module_values = []
    for i in range(0, kappa):
        best = None
        best_val = 0
        for pc in range(0, mod_count+1):
            val = reduced[i].subs([(p_ms[0, kappa - i - 1], T4prod * pc + base_prod + 1), (q_ms[0, kappa - i - 1], (mod_count - pc) * T4qual)])
            if val >= best_val:
                best = (pc, mod_count - pc)
                best_val = val

        assert not best is None
        reduced = [r.subs([(p_ms[0, kappa - i - 1], T4prod * best[0] + base_prod + 1), (q_ms[0, kappa - i - 1], best[1] * T4qual)]) for r in reduced]
        reduced_variances = [r.subs([(p_ms[0, kappa - i - 1], T4prod * best[0] + base_prod + 1), (q_ms[0, kappa - i - 1], best[1] * T4qual)]) for r in reduced_variances]
        best_setups.insert(0, best)
        best_module_values.insert(0, (T4prod * best[0] + base_prod + 1, best[1] * T4qual))
    
    return reduced, reduced_variances, best_setups, best_module_values

calculate_best_input_setup(4, Rational(0, 2))

([1,
  1/5,
  4249/72153,
  897021151/47203382487,
  56507468796165187/9265695305045843019],
 [1,
  3/23,
  1631259/60757099,
  12978930861567/2161964388670567,
  108488420298805297311/76174724615243583806711],
 [(2, 2), (2, 2), (3, 1), (4, 0), (4, 0)],
 [(3/2, 1/8), (3/2, 1/8), (7/4, 1/16), (2, 0), (2, 0)])

In [54]:
float(list(valuable_first_moments.values())[-1].subs([(q_rs[0, i], 4 * T4qual) for i in range(kappa)] + [(p_rs[0, i], 1) for i in range(kappa)] + [(q_ms[0, i], 5 * T4qual) for i in range(kappa)] + [(p_ms[0, i], 1 + Rational(1, 2)) for i in range(kappa-1)]+[(p_ms[0, kappa-1], 1 + Rational(1, 2) + 5 * T4prod)]))

0.47091418144393127

In [56]:
5 * T4qual

5/16

In [74]:
for v in list(input_valuable_first_moments.values())[:2]:
    display(v)

1

(-p_{m_4}*p_{r_4}*q_{m_4}*q_{r_4} + p_{m_4}*p_{r_4}*q_{r_4} + 4)/(-p_{m_4}*p_{r_4}*q_{m_4}*q_{r_4} + p_{m_4}*p_{r_4}*q_{m_4} + p_{m_4}*p_{r_4}*q_{r_4} - p_{m_4}*p_{r_4} + 4)

In [22]:
moment(binom, 1, 0) / m

1/4

In [ ]:
solve(Eq(q_ms[0], 1), [q_ms[0], q_ms[1]])

In [ ]:
A = MatrixSymbol('A', 2, 2)

# Define a 2x2 zero matrix
zero_matrix = Matrix.zeros(2, 2)

display(A[0, 1])

# Create a list of scalar equations for each element of the matrix A
equations = [Eq(A[i, j], zero_matrix[i, j]) for i in range(2) for j in range(2)]

display(equations)

# Solve the system of equations for A's elements
solution = solve(equations)

# Print the solution
print(solution)